In [1]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_4.npz', allow_pickle=False)
measurement = np.load('../../dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_symm_varspeed_1.npz', allow_pickle=False)

# measurement = np.load('../../dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_nomove_1.npz', allow_pickle=False)
# measurement = np.load('../../dataset/meas_asymm_reflector_1.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print(trainCIR.shape, testCIR.shape)

(6400, 15, 251, 2) (1600, 15, 251, 2)


In [2]:
def get64Samples(cirs):

    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    
    # Number of signalsd
    num_signals = real.shape[0]  # 3 in this case
    # print(f'num_signals: {num_signals}')
    # Initialize lists to store the focused samples
    imp_real_parts = []
    imp_imag_parts = []
    img_mag_parts = []
    
    for i in range(num_signals):
        # Calculate the magnitude
        magnitude = np.abs(real[i] + 1j * imag[i])
        
        # find the peak index
        peak_index = np.argmax(magnitude)
        
        # Calculate the start and end indices for the focused part
        start_index = max(0, peak_index - 32)
        end_index = min(magnitude.shape[0], peak_index + 32)
        
        # Extract the part of the signal around the peak
        real_part_focus = real[i, start_index:end_index]
        imag_part_focus = imag[i, start_index:end_index]
        mag_part_focus = magnitude[start_index:end_index]
        
        imp_real_parts.append(real_part_focus)
        imp_imag_parts.append(imag_part_focus)
        img_mag_parts.append(mag_part_focus)
        

    # Convert lists back to arrays for further processing if needed
    imp_real_parts = np.array(imp_real_parts)
    imp_imag_parts = np.array(imp_imag_parts)
    img_mag_parts = np.array(img_mag_parts)

    return imp_real_parts, imp_imag_parts, img_mag_parts



def getRealImaginaryMagnitudeParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    mag = np.abs(real + 1j * imag)
    return real, imag, mag

In [3]:
# SNR
def calculate_snr(real, imag, noise_power=1e-10):
    signal_power = np.sum(real**2 + imag**2, axis=1, keepdims=True)
    signal_power = np.maximum(signal_power, noise_power)  # Prevent very low or zero signal power
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Willison Amplitude
def calculate_willison_amplitude(feature, threshold=0.01):
    return np.sum(np.abs(np.diff(feature, axis=1)) > threshold, axis=1, keepdims=True)

# Decay Time Index
def calculate_decay_time_index(real, imag, p=0.5):
    h = np.sqrt(np.maximum(real**2 + imag**2, 1e-10))
    total_energy = np.sum(h**2, axis=1, keepdims=True)
    threshold = total_energy * p
    cumulative_energy = np.cumsum(h**2, axis=1)
    dti = np.argmax(cumulative_energy >= threshold, axis=1).reshape(-1, 1)  # Find the first index where cumulative energy exceeds threshold
    return dti

# Energy
def total_energy(real, imag):
    return np.sum(real**2 + imag**2, axis=1, keepdims=True)

In [4]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)


# Extract data for ALICE and BOB channels
alice_CIRs = trainCIR[:, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = trainCIR[:, eve_channel, :, :]  # Shape: (9797, 251, 2)

# Alice features:
#Real, imaginary, magnitude, phase
alice_train_real, alice_train_imag, alice_train_mag = getRealImaginaryMagnitudeParts(alice_CIRs)
# SNR
alice_train_snr = calculate_snr(alice_train_real, alice_train_imag)
# Willison Amplitude
alice_train_wp = calculate_willison_amplitude(alice_train_real)
# Decay Time Index
alice_train_dti = calculate_decay_time_index(alice_train_real, alice_train_imag)
# Energy
alice_train_energy = total_energy(alice_train_real, alice_train_imag)

# Features stack
alice_features = np.hstack((alice_train_real, alice_train_imag, alice_train_mag, alice_train_snr, alice_train_wp, alice_train_dti, alice_train_energy))


# EVE features - real, imaginary, magnitude, phase
eve_train_real, eve_train_imag, eve_train_mag = getRealImaginaryMagnitudeParts(eve_CIRs)
# SNR
eve_train_snr = calculate_snr(eve_train_real, eve_train_imag)
# Willison Amplitude
eve_train_wp = calculate_willison_amplitude(eve_train_real)
# Decay Time Index
eve_train_dti = calculate_decay_time_index(eve_train_real, eve_train_imag)
# Energy
eve_train_energy = total_energy(eve_train_real, eve_train_imag)

# Feature Stack
eve_features = np.hstack((eve_train_real, eve_train_imag, eve_train_mag, eve_train_snr, eve_train_wp, eve_train_dti, eve_train_energy))


# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice. (9797,)
eve_labels = np.ones(alice_features.shape[0])     # Label '1' for Eve. (9797,)

# Combine data and labels.
atoms = np.vstack((alice_features, eve_features))  # (19594, 192)
true_labels = np.hstack((alice_labels, eve_labels))  # (19594,)


# Combine data and labels for training
train_atoms = np.vstack((alice_features, eve_features))
train_labels = np.hstack((alice_labels, eve_labels))


# Apply PCA for Dimensionality Reduction
scaler = StandardScaler()
train_atoms_normalized = scaler.fit_transform(train_atoms)
print(f'atoms normalized : {train_atoms_normalized.shape}')

pca = PCA(n_components=250)
train_atoms_pca = pca.fit_transform(train_atoms_normalized)
print(f'atoms normalized pca : {train_atoms_pca.shape}')

D = train_atoms_pca.T

atoms normalized : (12800, 757)
atoms normalized pca : (12800, 250)


In [5]:
# Step 3: Extract Features for Test Data
alice_test_CIRs = testCIR[:, alice_channel, :, :]
eve_test_CIRs = testCIR[:, eve_channel, :, :]

# Alice test features:
# Real, imaginary, magnitude
alice_test_real, alice_test_imag, alice_test_mag = getRealImaginaryMagnitudeParts(alice_test_CIRs)
# SNR
alice_test_snr = calculate_snr(alice_test_real, alice_test_imag)
# Willison Amplitude
alice_test_wp = calculate_willison_amplitude(alice_test_real)
# Decay Time Index
alice_test_dti = calculate_decay_time_index(alice_test_real, alice_test_imag)
# Energy
alice_test_energy = total_energy(alice_test_real, alice_test_imag)
# Features stack
alice_test_features = np.hstack((alice_test_real, alice_test_imag, alice_test_mag, alice_test_snr, alice_test_wp, alice_test_dti, alice_test_energy))


# Eve test features:
# Real, imaginary, magnitude
eve_test_real, eve_test_imag, eve_test_magnitude = getRealImaginaryMagnitudeParts(eve_test_CIRs)
# SNR
eve_test_snr = calculate_snr(eve_test_real, eve_test_imag)
# Willison Amplitude
eve_test_wp = calculate_willison_amplitude(eve_test_real)
# Decay Time Index
eve_test_dti = calculate_decay_time_index(eve_test_real, eve_test_imag)
# Energy
eve_test_energy = total_energy(eve_test_real, alice_test_imag)
# Feature Stack
eve_test_features = np.hstack((eve_test_real, eve_test_imag, eve_test_magnitude, eve_test_snr, eve_test_wp, eve_test_dti, eve_test_energy))


alice_test_labels = np.zeros(alice_test_features.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_features.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = np.vstack((alice_test_features, eve_test_features))
test_labels = np.hstack((alice_test_labels, eve_test_labels))


test_atoms_normalized = scaler.transform(test_atoms)
test_atoms_pca = pca.transform(test_atoms_normalized)

In [6]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=20):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

In [7]:
# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual


In [8]:
# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


In [9]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms_pca:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
# print(predictions.shape)

accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 79.31%


In [10]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: (3200,)
tp: 1440
tn: 1098
fp: 502
fn: 160
MDR: 0.31375
FAR: 0.1
AR: 0.793125
